In [ ]:
'''
These scripts prepares raw downloaded CSV files for use with Open Tabulate.
It takes the raw directory as input and outputs to the processed directory.
The following steps are taken:
- Format latitude and longitude as seperate columns
- Converts format to CSV 
- Encodes CSV with utf-8
- Filters out data not related to health (eg police stations)

-Sam Lumley
Dec 2021

'''

In [3]:
import pandas as pd

# If necessary, install openpyxl and geopandas
# import sys
# !conda install --yes --prefix {sys.prefix} openpyxl

# %pip install geopandas

In [5]:
# Generate province folder structure

import os

folders=['ab','bc','mb','nb','nl','ns','nt','nu','on','pe','qc','sk','yt']

for items in folders:
    os.mkdir(items)


In [7]:
# AB Calgary covid 
# convert point to lat lon
# ignore empty rows

import pandas as pd

df=pd.read_csv('raw/AB_Calgary_covid_vaccination_centres.csv')

def strip_point(x):
    
    x=str(x)  
    if (x == 'nan'):
        t = [None,None]
        return t
    else:
        t=x.strip('POINT (')
        t=t.rstrip(')')
        return t.split()

LONGS=[]
LATS=[]
for i in df["POINT"]:
	LONGS.append(strip_point(i)[0])
	LATS.append(strip_point(i)[1])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('AB_Calgary_covid_vaccination_centres.csv')

In [69]:
# AB Clgary healthcare 
# - point to lat lon

import pandas as pd

df=pd.read_csv('AB_Calgary_healthcare_facilities.csv')

def strip_point(x):

    x=str(x)    
    t=x.strip('POINT (')
    t=t.rstrip(')')
    # 	print(t)
    return t.split()

LONGS=[]
LATS=[]
for i in df["location"]:
	LONGS.append(strip_point(i)[1])
	LATS.append(strip_point(i)[0])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('processed/AB_Calgary_healthcare_facilities.csv')

In [25]:
# Processing for BC
# utf-8 encode datasets

df=pd.read_csv('raw/BC_emergency_rooms.csv')
df.to_csv('processed/BC_emergency_rooms.csv')

df2=pd.read_csv('raw/BC_hospital.csv')
df2.to_csv('processed/BC_hospital.csv')

df3=pd.read_csv('raw/BC_pharmacies.csv')
df3.to_csv('processed/BC_pharmacies.csv')

df4=pd.read_csv('raw/BC_urgent_care.csv')
df4.to_csv('processed/BC_urgent_care.csv')

df5=pd.read_csv('raw/BC_walk-in_clinics.csv')
df5.to_csv('processed/BC_walk-in_clinics.csv')

df5=pd.read_csv('raw/ON_Guelph_healthcare_facilities.csv')
df5.to_csv('processed/ON_Guelph_healthcare_facilities.csv')

In [22]:
# Processing for Manitoba

# - read geojson, output csv
# - limit to hospitals
# - remove null values for geometry

import geopandas as gpd
import pandas as pd

#read geojson with geopandas into geodataframe
sc1=gpd.read_file('raw/MB_COVID-19_and_Flu_-_Vaccination_Sites.geojson')

df=pd.DataFrame(sc1)


def strip_point(x):
    
    x=str(x)  
    if (x == 'nan'):
        t = [None,None]
        return t
    elif x == 'None':
        t = [None,None]
        return t
    else:
        t=x.strip('POINT (')
        t=t.rstrip(')')
        return t.split() 


LONGS=[]
LATS=[]
for i in df.geometry:
	LONGS.append(strip_point(i)[0])
	LATS.append(strip_point(i)[1])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('processed/MB_COVID-19_and_Flu_-_Vaccination_Sites.csv')


In [37]:
# Processing for New Brunswick
# convert point to lat lon

df=pd.read_csv('raw/NB_nursing_homes.csv')

# df=df.loc[df["Facility Type"] != "Public Nursing Home"]

def strip_point(x):
	t=x.strip('(')
	t=t.rstrip(')')
#	t=t.strip(' (')
#	print(t)
	return t.split()

LONGS=[]
LATS=[]
for i in df["Location"]:
	LONGS.append(strip_point(i)[1])
	LATS.append(strip_point(i)[0])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('processed/NB_nursing_homes.csv')

In [32]:
# Processing for NL healthcare and hospital datasets
# convert from xls to csv

import openpyxl

# Assisted living

df = pd.read_excel("raw/NL_Hospital.xlsx", engine='openpyxl')
df.to_csv('processed/NL_Hospital.csv')

df2 = pd.read_excel("raw/NL_HealthCentre.xls", engine='openpyxl')
df2.to_csv('processed/NL_HealthCentre.csv')


In [41]:
# Processing for Nova Scotia
# Convert 'POINT' Geometry into lat and lon

import pandas as pd

df=pd.read_csv('raw/NS_hospital.csv')

def strip_point(x):

	x=str(x)    
	t=x.strip('POINT (')
	t=t.rstrip(')')
	return t.split()

LONGS=[]
LATS=[]
for i in df["the_geom"]:
	LONGS.append(strip_point(i)[0])
	LATS.append(strip_point(i)[1])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('processed/NS_hospital.csv')


In [14]:
# Processing for PEI Health Facilities data
# Turn 'Location" column into long and lat

df=pd.read_csv('raw/PE_healthcare_facilities.csv')

# df=df.loc[df["Facility Type"] != "Public Nursing Home"]

def strip_point(x):
	t=x.strip('(')
	t=t.rstrip(')')
#	t=t.strip(' (')
#	print(t)
	return t.split()

LONGS=[]
LATS=[]
for i in df["Location 1"]:
	LONGS.append(strip_point(i)[1])
	LATS.append(strip_point(i)[0])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('processed/PE_Hospitals.csv')

In [87]:
# Processing for NB Moncton 

#read geojson, output csv
#limit ourselves to hospitals
#Also turn 'POINT' Geometry into lat and lon

import geopandas as gpd
import pandas as pd

#read shapefile with geopandas into geodataframe

files = ['raw/NB_Moncton_Medical_Clinics', 'raw/NB_Moncton_Pharmacies', 'raw/NB_Moncton_Senior_Care_Facilities']

for file in files:

    filename = file + '.geojson'
    print(filename)
    sc1=gpd.read_file(filename)

    df=pd.DataFrame(sc1)


    def strip_point(x):

        x=str(x)    
        t=x.strip('POINT (')
        t=t.rstrip(')')
    # 	print(t)
        return t.split()

    LONGS=[]
    LATS=[]
    for i in df.geometry:
        LONGS.append(strip_point(i)[0])
        LATS.append(strip_point(i)[1])

    df["LONGITUDE"]=LONGS
    df["LATITUDE"]=LATS

    location = 'processed/' + file + '.csv'
    df.to_csv(location)

In [24]:
''' Processing ON Durham, Toronto clinics, 
Toronto covid test sites and York hospitals

read geojson, output csv
limit to hospitals
Also turn 'POINT' Geometry into lat and lon

'''

import geopandas as gpd
import pandas as pd

#read shapefile with geopandas into geodataframe

files = ['raw/ON_Durham_Healthcare_Facilities', 'raw/ON_Toronto_covid-19-immunization-clinics', 'raw/ON_Toronto_covid-19-testing-sites', 'raw/ON_York_Hospital']

for file in files:

    filename = file + '.geojson'
    print(filename)
    sc1=gpd.read_file(filename)

    df=pd.DataFrame(sc1)


    def strip_point(x):

        x=str(x)    
        t=x.strip('POINT (')
        t=t.rstrip(')')
    # 	print(t)
        return t.split()

    LONGS=[]
    LATS=[]
    for i in df.geometry:
        LONGS.append(strip_point(i)[0])
        LATS.append(strip_point(i)[1])

    df["LONGITUDE"]=LONGS
    df["LATITUDE"]=LATS

    location = 'processed/' + file + '.csv'
    df.to_csv(location)

raw/ON_Durham_Healthcare_Facilities.geojson


FileNotFoundError: [Errno 2] No such file or directory: 'processed/raw/ON_Durham_Healthcare_Facilities.csv'

In [95]:
# QC Gatineau 
# - convert POINT to lat lon

import pandas as pd

df=pd.read_csv('raw/QC_Gatineau_public_places_including_hospitals.csv')

def strip_point(x):

    x=str(x)    
    t=x.strip('POINT (')
    t=t.rstrip(')')
    # 	print(t)
    return t.split()

LONGS=[]
LATS=[]
for i in df["GEOM"]:
	LONGS.append(strip_point(i)[1])
	LATS.append(strip_point(i)[0])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df.to_csv('processed/QC_Gatineau_public_places_including_hospitals.csv')

In [100]:
# QC Quebec city 
# - convert POINT to lat lon

import pandas as pd

df=pd.read_csv('raw/QC_Quebec City_public_places_including_hospitals.csv')

def strip_point(x):

    x=str(x)    
    t=x.strip('POINT (')
    t=t.rstrip(')')
    # 	print(t)
    return t.split()

LONGS=[]
LATS=[]
for i in df["GEOMETRIE"]:
	LONGS.append(strip_point(i)[1])
	LATS.append(strip_point(i)[0])

df["LONGITUDE"]=LONGS
df["LATITUDE"]=LATS

df2 = df[df['DESCRIPTION'] == 'Hôpitaux']

df2.to_csv('processed/QC_Quebec City_public_places_including_hospitals.csv')

         ID                           NOM_TOPOGRAPHIE  NO_CIVIQUE  \
416  102269  Centre hospitalier de l'Université Laval      2705.0   
418  102267         Centre hospitalier Robert-Giffard         NaN   
696  102259                          Hôpital Chauveau     11999.0   
697  200528                        Hôpital Christ-Roi       900.0   
698  102237                 Hôpital de l'Enfant-Jésus      1401.0   
699  102263                Hôpital du Saint-Sacrement      1050.0   
700  200473                      Hôpital Jeffery Hale      1250.0   
701  102747                             Hôpital Laval         NaN   
703  102257           Hôpital Saint-François d'Assise        10.0   
741  102261                    L'Hôtel-Dieu de Québec        11.0   

                         RUE DESCRIPTION  SUPERFICIE  PERIMETRE  \
416        Boulevard Laurier    Hôpitaux           0          0   
418  Chemin de la Canardière    Hôpitaux           0          0   
696         Rue de l'Hôpital    Hôpitau

In [ ]:
'''

- calgary covid - point to lat lon /
- calgary healthcare - point to lat lon /
- moncton clinic - geojson /
- moncton pharm - geojson /
- moncton senior - geojson /
- durham - geojson
- hamilton hospital - fine
- ottawa hospital - fine
- toronto - 
- toronto - 
- york - 
- gatineau - 
- quebec city - 
- saskatoon - 

'''